## Backtesting
### Consideraciones sobre los dividendos

En este cuaderno se presentan las consideraciones que deberíamos tener presente, por un lado a la hora de interpretar resultados de simulaciones que incorporan directamente la re-inversión de dividendos y por otro lado la implementación de una estrategia de trading real que use series no ajustadas

___

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Datos
Tenemos la series de precios ajustada y no ajustada de la acción de ENAGAS.
Esta empresa reparte de forma regular 2 dividendos al año que podemos utilizar
para ver las diferencias respecto a una simulación auto-contenida en la serie
ajustada o una simulación real que acumula y reinvierte los dividendos.

Para más detalles ver info en la [web de la empresa](https://www.enagas.es/enagas/es/AccionistasEInversores/InformacionGeneral/Dividendos)

In [ ]:
stock_prices = pd.read_csv(
    '../data/ENG_prices.csv', 
    index_col=0,
    parse_dates=True)

In [ ]:
stock_prices

Dividendos de ENAGAS

In [ ]:
stock_divs = pd.read_csv('../data/ENG_dividends.csv', parse_dates=[0,1])
stock_divs

____

Comparación de serie ajustada y no ajustada

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
stock_prices.plot(ax=ax)

___
Obervamos que la fecha de pago es entre 2 y 4 sesiones posteriores a la fecha ex-dividendo. Aquí tenemos que considerar que:
 - Desde el punto de vista teórico, la serie ajustada puede reproducirse asumiendo el pequeños coste de financiar la compra el día del ex-dividendo. Esto es por ejemplo.
    - El día ex-dividendo se pide prestado el importe que recibirá el día de pago y se compran las acciones que se puedan a precio de ese día 
    - El día de pago se cobra el dividendo y se devuelve el prestamo
 - Desde un punto de vista práctico,  esto no es ejecutable sin una cuenta margen, por lo que tenemos que responder a la pregunta ¿**Cuánto coste implicito representa reinvertir los dividendo el día de pago**? 

___

- Hacemos la simulación de invertir inicialmente 100mil euros.
- Permitiremos invertir en fracciones de acciones para solo simular el efecto de comprar a precio distinto del ex-date

In [ ]:
capital = 100_000

In [ ]:
delta_shares = pd.Series(np.zeros(stock_prices.shape[0]), index=stock_prices.index)
init_shares  = capital/stock_prices.real.iloc[0]
delta_shares.iloc[0] = init_shares

In [ ]:
positions = delta_shares.cumsum()
positions.head()

In [ ]:
portvalue = positions * stock_prices.real

In [ ]:
portvalue.plot()

In [ ]:
delta_shares = pd.Series(np.zeros(stock_prices.shape[0]), index=stock_prices.index)
init_shares  = capital/stock_prices.real.iloc[0]
delta_shares.iloc[0] = init_shares
positions = delta_shares.cumsum()

for i, div_row in stock_divs.iterrows():
    pay_date = div_row['payment_date']
    ex_date = div_row['exdividend']
    curr_shares = positions.loc[ex_date]
    cash_div = curr_shares * div_row['dividend']
    
    new_shares = cash_div/stock_prices.real[pay_date]
    deserve_shares = cash_div/stock_prices.real[ex_date]
    delta_shares.loc[pay_date] = new_shares
    positions = delta_shares.cumsum()
    print(np.round(curr_shares, 2), cash_div, np.round(new_shares,2), np.round(deserve_shares, 2))

In [ ]:
delta_shares[delta_shares != 0]

In [ ]:
delta_shares.plot()

In [ ]:
portvalue = positions * stock_prices.real
portvalue.plot()

In [ ]:
relative_port = portvalue/portvalue.iloc[0]
relative_adjusted = stock_prices.adjusted/stock_prices.adjusted.iloc[0]

In [ ]:
relative_adjusted

In [ ]:
df_compare = pd.concat([relative_port, relative_adjusted], axis=1)
df_compare.columns = ['real_reinvest','adjusted']
df_compare.plot(figsize=(14,5))

In [ ]:
df_compare.loc['2018-06-28':'2018-07-10'].plot(figsize=(14,5))

In [ ]:
df_compare.tail()